In [1]:
from Bio import Entrez
import time
import subprocess

### TODO
- Put this in py file in the docker edirect environment
- Check and debug dockerfile to make image more accurate

In [2]:
def build_query():
    searchterm = input("Enter query searching term:")
    article_type = (' AND ("Clinical Trial"[PT] OR "Randomized Controlled Trial"[PT] OR "Meta-Analysis"[PT] '
                    'OR "Systematic Review"[PT] OR "Comparative Study"[PT] OR "Observational Study"[PT] '
                    'OR "Validation Study"[PT] OR "Case Reports"[PT] OR "Review"[PT])')
    full_query = (searchterm + article_type)
    full_query = full_query.replace("(", "\\(").replace(")", "\\)")

    sanitized_searchterm = searchterm #re.sub(r'[^a-zA-Z0-9_-]', '_', searchterm) -> need new docker image to install packages as regex
    return sanitized_searchterm, full_query


In [3]:
def create_script(sanitized_searchterm, full_query):
    script_filename = f"fetch_pubmed_{sanitized_searchterm.replace(' ', '_')}.sh"
    script_content = f""" #!/bin/bash
                    /root/edirect/esearch -db pubmed -query '{full_query}' | /root/edirect/efetch -format xml > {sanitized_searchterm.replace(' ', '_')}_pubmed_data.xml"""

    return script_filename, script_content
    
    


In [4]:
def fetch_data(script_filename, script_content):
    with open(script_filename, 'w') as file:
        file.write(script_content)

    
    #subprocess.run(f"docker exec -it edirect_container bash -c \"bash {script_filename}\"", shell=True, check=True)
    #subprocess.run(f"/bin/chmod +x {script_filename}", shell=True, check=True)
    #subprocess.run(f"bash {script_filename}'", shell=True, check=True)



In [5]:
Entrez.email = input("Enter emailadres:")
sanitized_searchterm, full_query = build_query()
script_filename, script_content = create_script(sanitized_searchterm, full_query)
fetch_data(script_filename, script_content)


Enter emailadres: michael.viroux@gmail.com
Enter query searching term: calcium


In [6]:
subprocess.run(["docker", "exec", "edirect_container", "bash", "-c", f"bash {script_filename}"], check=True)

CompletedProcess(args=['docker', 'exec', 'edirect_container', 'bash', '-c', 'bash fetch_pubmed_calcium.sh'], returncode=0)

In [ ]:
#subprocess.run(["docker", "exec", "edirect_container", "bash", "-c", "bash fetch_pubmed_calcium.sh"], check=True)
